### Project Overview and Import Modules

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
np.random.seed(0)
plt.style.use("ggplot")

import tensorflow as tf
print('Tensorflow version:', tf.__version__)
print('GPU detected:', tf.config.list_physical_devices('GPU'))

### Load and Explore the NER Dataset

*Essential info about tagged entities*:
- geo = Geographical Entity
- org = Organization
- per = Person
- gpe = Geopolitical Entity
- tim = Time indicator
- art = Artifact
- eve = Event
- nat = Natural Phenomenon

In [ ]:
data = pd.read_csv('ner_dataset.csv',encoding = 'latin1')
data = data.fillna(method='ffill')
data.head(20)

In [ ]:
print(data['Word'].nunique())
print(data['Tag'].nunique())

In [ ]:
words = list(set(data['Word'].values))
words.append('ENDPAD')
num_words = len(words)

In [ ]:
tags = list(set(data['Tag'].values))
num_tags = len(tags)

In [ ]:
print(num_words)
print(num_tags)

### Retrieve Sentences and Corresponsing Tags

In [ ]:
class SentenceGetter(object):
    def __init__(self,data):
        self.n_sent = 1
        self.data = data
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s['Word'].values.tolist(),
                                                          s['POS'].values.tolist(),
                                                          s['Tag'].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]

In [ ]:
getter = SentenceGetter(data)
sentences = getter.sentences

In [ ]:
sentences[0]

###  Define Mappings between Sentences and Tags

In [ ]:
words2idx = {w: i+1 for i,w in enumerate(words)}
tag2idx = {t: i for i , t, in enumerate(tags)}

In [ ]:
words2idx

###  Padding Input Sentences and Creating Train/Test Splits

In [ ]:
plt.hist([len(s) for s in sentences], bins = 50)
plt.show()

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

max_len = 50

x = [[words2idx[w[0]]for w in s]for s in sentences]
x = pad_sequences(maxlen = max_len,sequences = x,padding = 'post', value = num_words-1)

y = [[tag2idx[w[2]]for w in s]for s in sentences]
y = pad_sequences(maxlen=max_len,sequences = y,padding='post', value=tag2idx["O"])
y = [to_categorical(i, num_classes = num_tags)for i in y]

In [ ]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x,y, test_size=0.1, random_state=1)

###  Build and Compile a Bidirectional LSTM Model

In [ ]:
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense
from tensorflow.keras.layers import TimeDistributed, SpatialDropout1D, Bidirectional

In [ ]:
input_word = Input(shape=(max_len,))
model = Embedding(input_dim=num_words, output_dim=max_len, input_length=max_len)(input_word)
model = SpatialDropout1D(0.1)(model)
model = Bidirectional(LSTM(units=100,return_sequences=True,recurrent_dropout=0.1))(model)
out = TimeDistributed(Dense(num_tags,activation='softmax'))(model)
model = Model(input_word, out)
model.summary()

In [ ]:
model.compile(optimizer="adam",loss='categorical_crossentropy',metrics=['accuracy'])

###  Train the Model

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from livelossplot.tf_keras import PlotLossesCallback

In [ ]:
early_stopping = EarlyStopping(monitor='val_accuracy', patience=1, verbose=0, mode='max',restore_best_weights=False)
callbacks = [PlotLossesCallback(),early_stopping]
history = model.fit(
    x_train , np.array(y_train),
    validation_split=0.2,
    batch_size=32,
    epochs = 3,
    verbose = 1,
    callbacks = callbacks
)

###  Evaluate Named Entity Recognition Model

In [ ]:
model.evaluate(x_test,np.array(y_test))

In [ ]:
i = np.random.randint(0,x_test.shape[0])
p = model.predict(np.array([x_test[i]]))
p = np.argmax(p,axis=-1)

y_true = np.argmax(np.array(y_test), axis=-1)[i]
print("{:15}{:15}\t {}\n".format('Word',"True","Pred"))
print("."*30)
for w,true,pred in zip(x_test[i],y_true,p[0]):
    print("{:15}{}\t{}".format(words[w-1],tags[true],tags[pred]))